## Integer arithmetic

Many quantum algorithms need to evaluate classical functions on a superposition of inputs. An example is Shor's algorithm, which requires the mapping $|x\rangle|0\rangle\mapsto|x\rangle|a^x\text{ mod }N\rangle$ to be implemented as a quantum operation.

To keep users from having to implement such functions from scratch, the canon ([`Microsoft.Quantum.Canon`](https://docs.microsoft.com/qsharp/api/canon/microsoft.quantum.canon)) contains arithmetic functionality such as integer adders.

Let's write a wrapper for one of these adders, that of [Takahashi *et al.*](https://arxiv.org/abs/0910.2530). This will allow us to quickly exchange it for a different implementation if need be.

In [1]:
open Microsoft.Quantum.Arithmetic;

operation Add(qint1 : LittleEndian, qint2 : LittleEndian, carry : Qubit) : Unit {
    Message("[Using the adder by Takahashi et al. (arXiv:0910.2530)]");
    RippleCarryAdderTTK(qint1, qint2, carry);
}

Add

### Running the adder

We can now use this wrapper in our code as a subroutine. As an example, let's write an operation that initializes two quantum integers to classical values (e.g., 4 and 5) before calling our `Add` operation.

In [2]:
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Measurement;

operation TestAddition() : Int {
    let firstNumber = 4;
    let secondNumber = 5;
    let numBits = 3;

    mutable result = 0;
    Message($"Calculating {firstNumber} + {secondNumber}...");
    using ((register1, register2, carry) = (Qubit[numBits], Qubit[numBits], Qubit())) {
        let qint1 = LittleEndian(register1);
        let qint2 = LittleEndian(register2);
        
        // initialize the two quantum integers:
        ApplyXorInPlace(firstNumber, qint1);
        ApplyXorInPlace(secondNumber, qint2);
        
        // perform addition
        Add(qint1, qint2, carry);
        
        // and measure the result
        let mCarry = (MResetZ(carry) == One) ? 1 | 0;
        set result = 2^numBits * mCarry + MeasureInteger(qint2);
        ResetAll(register1 + register2);
    }

    // Return the measured result
    return result;
}

TestAddition

We can now test our adder using the simulator:

In [3]:
%simulate TestAddition 

Calculating 4 + 5...
[Using the adder by Takahashi et al. (arXiv:0910.2530)]


9

### Exchanging implementations
Furthermore, we can quickly change the underlying adder implementation, e.g., to that of [Cuccaro *et al.*](https://arxiv.org/abs/quant-ph/0410184):

In [4]:
open Microsoft.Quantum.Arithmetic;

operation Add(qint1 : LittleEndian, qint2 : LittleEndian, carry : Qubit) : Unit {
    Message("[Using the adder by Cuccaro et al. (arXiv:0410184v1)]");
    RippleCarryAdderCDKM(qint1, qint2, carry);
}

Add

Let's run our `TestAddition` operation again on the simulator:

In [5]:
%simulate TestAddition

Calculating 4 + 5...
[Using the adder by Cuccaro et al. (arXiv:0410184v1)]


9

### Version information
This notebook was last executed with:

In [6]:
%version

Component,Version
iqsharp,0.9.1908.2906
Jupyter Core,1.1.14623.0
